
add an env file with:
OPENAI_API_KEY = ...
LANGCHAIN_TRACING_V2=true
LANGCHAIN_API_KEY=...

to activate env cd to the project folder and run :

source env/bin/activate

In [ ]:
# pip install -r requirements.txt

In [8]:
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [9]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [10]:
chain = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)


In [17]:
image = encode_image("3.png")


msg = chain.invoke(
    [   AIMessage(
        content="You are a useful bot that is especially good at OCR from images for math formulas and graphs"
    ),
        HumanMessage(
            content=[
                {"type": "text", "text": "solve for all questions"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image}"
                    },
                },
            ]
        )
    ]
)
print(msg.content)

The given image contains three problems, all of which are asking to use Simpson's rule to approximate the integral from x = 3 to x = 5 of the function f(x). The values of f(x) at x = 3, 4, and 5 are provided for each problem.

Simpson's rule is given by the formula:

\[ \int_{a}^{b} f(x) \, dx \approx \frac{\Delta x}{3} [f(x_0) + 4f(x_1) + f(x_2)] \]

where \( \Delta x = \frac{b - a}{n} \) (and n is even, which is the case here with n = 2).

Let's apply Simpson's rule to each problem:

(a) f(x) at x = 3, 4, 5 are 7, 4, and 10, respectively.
\[ \Delta x = \frac{5 - 3}{2} = 1 \]
\[ \approx \frac{1}{3} [7 + 4(4) + 10] \]
\[ \approx \frac{1}{3} [7 + 16 + 10] \]
\[ \approx \frac{1}{3} [33] \]
\[ \approx 11 \]

(b) f(x) at x = 3, 4, 5 are 2.3, 4.1, and 2.6, respectively.
\[ \Delta x = \frac{5 - 3}{2} = 1 \]
\[ \approx \frac{1}{3} [2.3 + 4(4.1) + 2.6] \]
\[ \approx \frac{1}{3} [2.3 + 16.4 + 2.6] \]
\[ \approx \frac{1}{3} [21.3] \]
\[ \approx 7.1 \]

(c) f(x) at x = 3, 4, 5 are -8, -4, and -3,